In [ ]:
import pandas as pd
import numpy as np
import glob
import math

In [ ]:
!conda install -c ibmdecisionoptimization docplex

from docplex import *
from cplex import *
import docplex.mp.model as cpx

# Select Normalization Technique

In [ ]:
# Get Normalization Scales

Y = []
for x in range(10):
    #y = 10*math.exp(-0.5*x)  # Exponential Normalization 0.5
    #y = 10 - x               # Linear Normalization 
    #y = 10*math.exp(-0.2*x)  # Exponential Normalization 0.2
    #y = 10*math.exp(-2*x)     # Exponential Normalization 2
    #y = 1                     # Same Preference 
    y = 11-math.exp(0.24*x)  #Exponential Concave
    Y.append(y)

# Data Pre-Processing

In [ ]:
path = r'' # use your path
all_files = glob.glob(path + "/*.csv")

d = {}
set_students = []

for filename in all_files:
    student_name = filename.split('\\')[-1].split(".")[0]
    df = pd.read_csv(filename, index_col=None, header=0)
    df.columns = ['Day','From','To','Pref','Order']
    d[student_name] = df
    set_students.append(student_name)
    
n_students = len(set_students)

In [ ]:
totalpoints = dict(zip(set_students,[13,7,10,14,15,9,10,8,8,8,13,6,9,15,8,10,9,6]))

In [ ]:
# Data Manipulation
for x in set_students:
    d[x]["Day"]= d[x]["Day"].str.title() 
    d[x]['length'] = d[x]['To'] - d[x]['From']
    preflength = len(d[x]['Day'])
    d[x]['Scale'] = Y[0:preflength]
    d[x]['Step1'] = d[x]['length']*d[x]['Scale']
    total = 0 
    for i in range(preflength):
      total = total + d[x]['Step1'][i]
    d[x]['Total1'] = total
    d[x]['Step2'] = ( d[x]['Step1'] / d[x]['Total1'] ) * totalpoints[x]
    d[x]['Preference'] = d[x]['Step2'] / d[x]['length']

In [ ]:
# Create New Dataframe
New = {}
for x in set_students:
    New[x] = d[x][['Day', 'From', 'To', 'Preference','Order']].copy()

In [ ]:
# Value Assignment 
preference = {}
for x in set_students:
    df_np = New[x].values
    data = {'Day':['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']}
    master = pd.DataFrame(data)
    shift = [*range(1,17,1)]
    for i in shift:
      master[i] = 0.0;
    master.set_index('Day', inplace=True)
    for j in range(df_np.shape[0]):
        key = df_np[j,0].title()
        from_idx = df_np[j,1]
        to_idx = df_np[j,2]
        value = df_np[j,3]
        rng = list(range(from_idx, to_idx))
        master.loc[key,rng] = value
        preference[x] = master

In [ ]:
for x in set_students:
    preference[x].to_excel('{}.xlsx'.format(x))

# Scheduling Model 

In [ ]:
n_days, n_shifts = preference[list(preference.keys())[0]].shape
# Sets
set_days = preference[list(preference.keys())[0]].index
set_shifts = preference[list(preference.keys())[0]].columns

In [ ]:
max_shifts = [13,7,10,14,15,17,10,8,8,8,13,6,9,15,8,10,9,6]
required = {(j,k): 2 for j in set_days for k in set_shifts}
max_shift = dict(zip(set_students,max_shifts))
shift_preference = {(i,j,k) : preference['{}'.format(i)][k][j] for i in set_students for j in set_days for k in set_shifts}

In [ ]:
# Initialize Model
opt_model = cpx.Model(name="MIP Model")

In [ ]:
# Decision Variable
x_vars = {(i,j,k): opt_model.binary_var(name="x_{0}_{1}_{2}".format(i,j,k)) 
for i in set_students for j in set_days for k  in set_shifts}

In [ ]:
# Constraints 

# Max Shift / Day Constraint
maxshiftperday = {(i,j) : 
opt_model.add_constraint(
ct=opt_model.sum(x_vars[i,j,k] for k in set_shifts) <= 8,
ctname="maxshiftperday_{0}_{1}".format(i,j))
    for i in set_students for j in set_days}

# Available Positions 
availableposition = {(j,k) : 
opt_model.add_constraint(
ct=opt_model.sum(x_vars[i,j,k] for i in set_students) <= required[j,k],
ctname="availableposition_{0}_{1}".format(j,k))
    for j in set_days for k in set_shifts}

# Shifts per week
shiftsperweek = {i : 
opt_model.add_constraint(
ct=opt_model.sum(x_vars[i,j,k] for j in set_days for k in set_shifts) <= max_shift[i],
ctname="shiftsperweek_{0}".format(i))
    for i in set_students}

# Minimum one shift per student 
minshiftperstudent = {i : 
opt_model.add_constraint(
ct=opt_model.sum(x_vars[i,j,k] for j in set_days for k in set_shifts) >= 1,
ctname="minshiftperstudent_{0}".format(i))
    for i in set_students}

# Proper Assignment
for i in set_students:
  for j in set_days:
    for k in set_shifts:
      if shift_preference[i,j,k]==0:
        assignment = {(i,j,k): opt_model.add_constraint(ct = opt_model.add(x_vars[i,j,k] == 0),ctname='assignment_{0}_{1}_{2}'.format(i,j,k))}

In [ ]:
# Objective Function
pref_max = opt_model.sum(x_vars[i,j,k] * shift_preference[i,j,k] 
                          for i in set_students for j in set_days for k in set_shifts)

In [ ]:
#for maximization
opt_model.maximize(pref_max)
opt_model.solve(url=None,key=None)

In [ ]:
res = opt_model.solution.get_value_df(x_vars)

In [ ]:
#print(opt_model.solution)

# Schedule Output

In [ ]:
# Final Schedule 
res = opt_model.solution.get_value_df(x_vars)
fin = res[res['value']==1].sort_values(['key_2','key_3']).reset_index().drop(['index','value'],axis=1)
fin.columns = ['Student_Name','Day','Shift']
fin = fin.reset_index()
fin = fin.groupby(['Day', 'Shift'])

result = fin['Student_Name'].unique()
result = result.reset_index()
schedule = pd.DataFrame(index=set_days,columns=set_shifts) 
for x in result.index:
    i = result['Day'][x]
    j = result['Shift'][x]
    schedule[j][i] = result['Student_Name'][x]
schedule

# Result Statistics 

In [ ]:
# Percentage of Shift Allotment
def percentshift():
  x = 0
  for i in set_students:
    for j in set_days:
      for k in set_shifts:
        x = x + opt_model.sum(x_vars[i,j,k])
  percentageshift = x / (n_days * n_shifts)
  return percentageshift;

# Total Shift Allotment
def totalshift():
  x = 0
  for i in set_students:
    for j in set_days:
      for k in set_shifts:
        x = x + opt_model.sum(x_vars[i,j,k])
  return x;

# SHift Allotment per Student
def shiftperstudent():
  studentdict = {}
  for i in set_students:
    x = 0
    for j in set_days:
      for k in set_shifts:
        x = x + opt_model.sum(x_vars[i,j,k])
    studentdict[i] = x
  return studentdict;

In [ ]:
# Add KPI's
opt_model.add_kpi(totalshift(), "Total Shift by function")    # Total Shift Allotment 
for i in set_students:                                        # Shift Allotment per Student
  opt_model.add_kpi( (shiftperstudent()[i]), "Number of Shift Allotted to {}".format(i))
for i in set_students:                                        # Percent of Shift Allotment per Student
  opt_model.add_kpi( (shiftperstudent()[i] / max_shift[i]), "Percent of Shift Allotted to {}".format(i))
opt_model.add_kpi( percentshift(), "Percent of Shift Allotted") # Percent of total Shift Allotment

In [ ]:
opt_model.report_kpis()

In [ ]:
solutiondict = opt_model.kpis_as_dict()

In [ ]:
solutiondict